In [1]:
import ee
import geemap
import datetime

In [4]:
ee.Initialize(project='tidal-heading-463908-a6')
print("GEE 연결 성공!")

GEE 연결 성공!


In [18]:
start_date = '2015-01-01'
end_date = '2023-12-31'
dates = pd.date_range(start=start_date, end=end_date)

In [15]:
collection = ee.ImageCollection('MODIS/061/MCD19A2_GRANULES') \
    .filterDate(start_date, end_date) \
    .select('Optical_Depth_047')

In [13]:
import pandas as pd

In [ ]:
# 중국 경계 (대략)
china = ee.Geometry.Rectangle([73.5, 18.0, 135.1, 53.6])

# AOD(MODIS)
collection = ee.ImageCollection('MODIS/061/MCD19A2_GRANULES') \
    .filterDate(start_date, end_date) \
    .select('Optical_Depth_047')

# 날짜별 평균값 추출 함수
def get_mean_aod(date_str):
    image = collection.filterDate(date_str, ee.Date(date_str).advance(1, 'day')).mean().clip(china)
    stat = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=china,
        scale=1000,
        maxPixels=1e13
    )
    return stat.get('Optical_Depth_047').getInfo()

aod_list = []
for date in dates:
    date_str = date.strftime('%Y-%m-%d')
    try:
        value = get_mean_aod(date_str)
        aod_list.append({'date': date_str, 'AOD': value})
        print(f"{date_str}: {value}")
    except Exception as e:
        print(f"Error on {date_str}: {e}")
        aod_list.append({'date': date_str, 'AOD': None})

df = pd.DataFrame(aod_list)
df.to_csv('china_aod_daily.csv', index=False)
print("✅ 완료: china_aod_daily.csv 저장됨")   

2015-01-01: 207.21374850561097
2015-01-02: 205.82700693385578
2015-01-03: 248.65025115175732
2015-01-04: 330.15392579364726
2015-01-05: 237.30570667939463
2015-01-06: 247.883067797476
2015-01-07: 324.96644590572663
2015-01-08: 322.7650756702785
2015-01-09: 279.5396553388057
2015-01-10: 266.7603073811329
2015-01-11: 271.21301125415664
2015-01-12: 286.6147317130171
2015-01-13: 366.8334334758415
2015-01-14: 372.3153669661614
2015-01-15: 352.9697119825902
2015-01-16: 390.108320347915
2015-01-17: 352.8962350989279
2015-01-18: 325.6280249563729
2015-01-19: 298.80309152718786
2015-01-20: 293.6851012892623
2015-01-21: 319.9458104198648
2015-01-22: 315.7933194003553
2015-01-23: 379.4042100091872
2015-01-24: 292.11411321456177
2015-01-25: 296.6692180808297
2015-01-26: 304.08286746813593
2015-01-27: 308.4006379873289
2015-01-28: 332.9112826782604
2015-01-29: 376.01218802986193
2015-01-30: 344.3999061724845
2015-01-31: 312.15655574709047
2015-02-01: 341.4922747455865
2015-02-02: 327.846684346393
2

In [ ]:
import pandas as pd

df = pd.read_csv('china_aod_daily.csv')

# 스케일 보정 (단위: * 0.001)
df['AOD_scaled'] = df['AOD'] * 0.001

# 황사 여부 판단
df['황사_가능성'] = df['AOD_scaled'].apply(lambda x: '황사' if pd.notnull(x) and x >= 0.4 else '양호')

print(df.head())

df.to_csv('china_aod_scaled.csv', index=False)
print("✅ 완료: china_aod_scaled.csv 저장됨")

         date         AOD  AOD_scaled 황사_가능성
0  2015-01-01  207.213749    0.207214     양호
1  2015-01-02  205.827007    0.205827     양호
2  2015-01-03  248.650251    0.248650     양호
3  2015-01-04  330.153926    0.330154     양호
4  2015-01-05  237.305707    0.237306     양호
✅ 완료: china_aod_scaled.csv 저장됨


In [ ]:
# 날짜별 최댓값 추출 함수
def get_max_aod(date_str):
    image = collection.filterDate(date_str, ee.Date(date_str).advance(1, 'day')).mean().clip(china)
    stat = image.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=china,
        scale=1000,
        maxPixels=1e13
    )
    return stat.get('Optical_Depth_047').getInfo()

#추출
aod_list = []
for date in dates:
    date_str = date.strftime('%Y-%m-%d')
    try:
        value = get_max_aod(date_str)
        aod_list.append({'date': date_str, 'AOD_max': value})
        print(f"{date_str}: {value}")
    except Exception as e:
        print(f"Error on {date_str}: {e}")
        aod_list.append({'date': date_str, 'AOD_max': None})

# 저장
df = pd.DataFrame(aod_list)
df.to_csv('china_aod_max_daily.csv', index=False)
print("✅ 완료: china_aod_max_daily.csv 저장됨")

2015-01-01: 4000
2015-01-02: 4000
2015-01-03: 3727
2015-01-04: 4000
2015-01-05: 4000
2015-01-06: 5202
2015-01-07: 4087
2015-01-08: 4431
2015-01-09: 4656
2015-01-10: 4000
2015-01-11: 4000
2015-01-12: 4287
2015-01-13: 4113
2015-01-14: 4599
2015-01-15: 5043
2015-01-16: 4075
2015-01-17: 4255
2015-01-18: 4000
2015-01-19: 4175
2015-01-20: 4000
2015-01-21: 4741
2015-01-22: 4000
2015-01-23: 5861
2015-01-24: 4213
2015-01-25: 4334
2015-01-26: 4047
2015-01-27: 5824
2015-01-28: 5090
2015-01-29: 5301
2015-01-30: 5142
2015-01-31: 8751
2015-02-01: 6110
2015-02-02: 4000
2015-02-03: 4610
2015-02-04: 5796
2015-02-05: 5658
2015-02-06: 5085
2015-02-07: 8259
2015-02-08: 4000
2015-02-09: 4000
2015-02-10: 4812
2015-02-11: 4000
2015-02-12: 4000
2015-02-13: 4000
2015-02-14: 5748
2015-02-15: 5243
2015-02-16: 6658
2015-02-17: 5712
2015-02-18: 5401
2015-02-19: 5755
2015-02-20: 5382
2015-02-21: 5638
2015-02-22: 5682
2015-02-23: 4765
2015-02-24: 5334
2015-02-25: 4926
2015-02-26: 5750
2015-02-27: 5843
2015-02-28: 60